# Explore Minerva simulations on nimbus

Quick overview of the simulations generated by the `multi_round_sim` branch of r2b2,
and the `multi_round_sim` module, and the `parse_sims` module that parses its output.

The multi_round_sim.py code needs a lot of tidying and parameterization....

Note that this notebook can be used from home via an ssh tunnel to nimbus:

Run on nimbnode4, and note the token authentication query params it offers:
`jupyter notebook`

Run at home:
`ssh -NL 7777:localhost:8888 nimbnode4 &`

open at home, to authenticate:
http://localhost:8888/?token=.....

and finally e.g.

http://localhost:7777/notebooks/simout/explore_nimbus_simulations.ipynb


In [72]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 13, 8
%matplotlib inline

In [73]:
sys.path.insert(0, '/home/nmcburnett/bayes/r2b2/src')

In [74]:
import parse_sims

## Run a new simulation

In [85]:
!RANDSEED=17 taskset -c 2 python -u ~/multi_round_sim.py > test.out

## Look at some recent simultations of contests with multiple pairs of winner and loser.

In [ ]:
!head -70 m2.5-r10-s50-0

This one has a 2.5% margin, a risk limit of 10%,
and uses the Minerva first round stopping probability of 70%
(which is of course, calculated only based on the closest margin).

In [16]:
sim = "m2.5-r10-s70-0"

In [17]:
!head -40 $sim

Name: athena
Version: 0.8.2
Summary: UNKNOWN
Home-page: https://github.com/filipzz/athena
Author: Grant McClearn, Sarah Morin, Neal McBurnett, Poorvi Vora, Filip Zagorski
Author-email: filip.zagorski@gmail.com
License: UNKNOWN
Location: /home/nmcburnett/Envs/athena-new/lib/python3.8/site-packages
Requires: requests, scipy
Required-by: 

7cdbdaf157710c125dedcafa878da7ce3c042364 2.5% margin

On branch multi_round_sim
Your branch is ahead of 'origin/multi_round_sim' by 42 commits.
  (use "git push" to publish your local commits)

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	src/parse_sims.py

nothing added to commit but untracked files present (use "git add" to track)

seed=(834, 0), risk_limit=0.1, margin=2.50%, num_candidates=3, num_winners=1, ballots=1000, tally={'A': 350, 'B': 325, 'C': 325}, winners=A, trueprobs=array([0.35 , 0.325, 0.325]), SPROB=0.7
  {'relevant_sample_size': 4337, 'sample': [1498, 1438, 1401], 'p_value': 0.16824504888836328, 

In [18]:
!tail -n 2 $sim

100.00% (100000/100000) of the audits passed:
Round size counter: [(1, 55399), (2, 34173), (3, 8647), (4, 1639), (5, 139), (6, 3)]


Thus, first round completion was only 55.399% in this special case, rather than 70%.

Subsequent rounds were each calculated with a 1.5 MINERVA_MULTIPLE. Here, most of the rest of the audits done by the 2nd or third round, with the following stopping probabilities:

In [36]:
round_tag = 'Round size counter: '

In [60]:
results = !tail -10 $sim
rounds = [r[1] for r in eval(results.grep(round_tag)[0][len(round_tag):])]

In [59]:
remaining = sum(rounds)
for round_size in rounds:
    print(f'{round_size / remaining:.2%} for {round_size=}, {remaining=}')
    remaining -= round_size

55.40% for round_size=55399, remaining=100000
76.62% for round_size=34173, remaining=44601
82.92% for round_size=8647, remaining=10428
92.03% for round_size=1639, remaining=1781
97.89% for round_size=139, remaining=142
100.00% for round_size=3, remaining=3


In [67]:
def sprob_results(sim):
    results = get_ipython().getoutput(f'tail -10 {sim}')
    #results = !tail -10 $sim
    rounds = [r[1] for r in eval(results.grep(round_tag)[0][len(round_tag):])]
    remaining = sum(rounds)
    for round_size in rounds:
        print(f'{round_size / remaining:.2%} for {round_size=}, {remaining=}')
        remaining -= round_size

In [69]:
nsim = 'm2.5-r10-s50-3'

In [70]:
!tail -n 2 $nsim

seed=(19, 25929), risk_limit=0.1, margin=2.50%, num_candidates=3, num_winners=1, ballots=1000, tally={'A': 350, 'B': 325, 'C': 325}, winners=A, trueprobs=array([0.35 , 0.325, 0.325]), SPROB=0.5
  {'relevant_sample_size': 2962, 'sample': [1045, 937, 980], 'p_value': 0.1298305016174012, 'round': 1, 'cpu': 0.18959}


In [68]:
sprob_results(nsim)

IndexError: list index out of range

## Use parse_sims to read in round-by-round details

In [75]:
acc = parse_sims.parse(sim)

In [76]:
parse_sims.show_minmax(acc)

len(rows)=200025 rows in schema=frozenset()
len(rows)=100000 rows in schema=frozenset({'margin', 'SPROB', 'winners', 'tally', 'risk_limit', 'num_candidates', 'num_winners', 'seed', 'ballots', 'trueprobs'})
len(rows)=156955 rows in schema=frozenset({'cpu', 'round', 'p_value', 'relevant_sample_size', 'sample'})
Lowest 10 pvalues:
{'relevant_sample_size': 32934, 'sample': [11738, 10603, 10593], 'p_value': 1.1543050065313666e-09, 'round': 6, 'cpu': 27.93565}
{'relevant_sample_size': 14637, 'sample': [5313, 4615, 4709], 'p_value': 4.777999106260529e-09, 'round': 4, 'cpu': 2.80447}
{'relevant_sample_size': 32934, 'sample': [11592, 10669, 10673], 'p_value': 8.40389522262978e-09, 'round': 6, 'cpu': 14.17537}
{'relevant_sample_size': 21956, 'sample': [7846, 7074, 7036], 'p_value': 1.4398345851375042e-08, 'round': 5, 'cpu': 6.43715}
{'relevant_sample_size': 14637, 'sample': [5309, 4656, 4672], 'p_value': 1.7409157578029412e-08, 'round': 4, 'cpu': 3.05856}
{'relevant_sample_size': 21956, 'sample'

In [79]:
fs0 = frozenset({'margin', 'SPROB', 'winners', 'tally', 'risk_limit', 'num_candidates', 'num_winners', 'seed', 'ballots', 'trueprobs'})

In [82]:
fs1 = frozenset({'cpu', 'round', 'p_value', 'relevant_sample_size', 'sample'})

In [80]:
df = pd.DataFrame(acc[fs0])

In [81]:
df

,seed,risk_limit,margin,num_candidates,num_winners,ballots,tally,winners,trueprobs,SPROB
0,"(834, 0)",0.1,2.50%,3,1,1000,"{'A': 350, 'B': 325, 'C': 325}",A,"array([0.35 , 0.325, 0.325])",0.7\n
1,"(834, 1)",0.1,2.50%,3,1,1000,"{'A': 350, 'B': 325, 'C': 325}",A,"array([0.35 , 0.325, 0.325])",0.7\n
2,"(834, 2)",0.1,2.50%,3,1,1000,"{'A': 350, 'B': 325, 'C': 325}",A,"array([0.35 , 0.325, 0.325])",0.7\n
3,"(834, 3)",0.1,2.50%,3,1,1000,"{'A': 350, 'B': 325, 'C': 325}",A,"array([0.35 , 0.325, 0.325])",0.7\n
4,"(834, 4)",0.1,2.50%,3,1,1000,"{'A': 350, 'B': 325, 'C': 325}",A,"array([0.35 , 0.325, 0.325])",0.7\n
...,...,...,...,...,...,...,...,...,...,...
99995,"(834, 99995)",0.1,2.50%,3,1,1000,"{'A': 350, 'B': 325, 'C': 325}",A,"array([0.35 , 0.325, 0.325])",0.7\n
99996,"(834, 99996)",0.1,2.50%,3,1,1000,"{'A': 350, 'B': 325, 'C': 325}",A,"array([0.35 , 0.325, 0.325])",0.7\n
99997,"(834, 99997)",0.1,2.50%,3,1,1000,"{'A': 350, 'B': 325, 'C': 325}",A,"array([0.35 , 0.325, 0.325])",0.7\n
99998,"(834, 99998)",0.1,2.50%,3,1,1000,"{'A': 350, 'B': 325, 'C': 325}",A,"array([0.35 , 0.325, 0.325])",0.7\n


In [83]:
dfcpu = pd.DataFrame(acc[fs1])

In [84]:
dfcpu

,relevant_sample_size,sample,p_value,round,cpu
0,32934,"[11738, 10603, 10593]",1.154305e-09,6,27.93565
1,14637,"[5313, 4615, 4709]",4.777999e-09,4,2.80447
2,32934,"[11592, 10669, 10673]",8.403895e-09,6,14.17537
3,21956,"[7846, 7074, 7036]",1.439835e-08,5,6.43715
4,14637,"[5309, 4656, 4672]",1.740916e-08,4,3.05856
...,...,...,...,...,...
156950,14637,"[5001, 4993, 4643]",7.922657e+00,4,2.84250
156951,9758,"[3270, 3161, 3327]",9.417983e+00,3,1.14852
156952,14637,"[4970, 4855, 4812]",9.959944e+00,4,2.80831
156953,9758,"[3289, 3125, 3344]",1.006220e+01,3,1.23853
